# Building AI for BlackJack Game

This notebook is for explorartion purposes. I keep this notebook as a journal.

- This idea has been started with a conversation that I have made with my nephew. He asked me if we could build an AI agent that plays the best moves for blackjack. After some arguing on the topic, we have agreed that it would be overkill to train a deep net. Hence there were already best posible moves avaible. However this table only keeps the best action to play in a total matrix of player and dealer. So we have agreed to at least to construct a q-value table via reinforcement learning. We are expecting to construct a q-value table that will consists of the best move to to make (Hit,Double,Stand) in percentages. Our intension is to keep it simple so we will not implement the full rrules of the game. We will consider Ace as 1 not 11. I've learned that this is called hard totals. The avaible best moves are as follows:

![Best Action Table](best_moves_in_hard_totals.jpeg)

- So our plan is to put the percentages of each action for each state. (Q-Values)
- We are also aware of [this](https://towardsdatascience.com/reinforcement-learning-solving-blackjack-5e31a7fb371f) page. We may check it out in the future. But for learning purposes I will check it out if I geet stuck of my own.

28.05.2020

- I start right away coding the environment, I need to remember my other projects. First I will copy my abstract methods here.
- So while checking my other [projects](https://github.com/hakanonal/geodashml) I need to decide the dimensions of the table. 
    - 1 dimension is for the playes's total card.
    - 1 dimension is for the dealers open card.
    - and one dimension is for the action. which is for simplicity Hit or Stand.
- So the shape of the table is (21,10,2)
- Ok very simple q-tabled agent has been completed.
- action space 0 is stand and 1 is hit
- Ok I am currentlly stuck at play method of environment class. I need to think thourgh how should construct reward. Since getting the hihgest number hould not be promoted. But I need to give reward to get more that dealer.

In [2]:
import numpy as np
import random


class agent:

    def __init__(self,discount=0.95,exploration_rate=0.9,decay_factor=0.9999, learning_rate=0.1):
        self.discount = discount # How much we appreciate future reward over current
        self.exploration_rate = exploration_rate # Initial exploration rate
        self.decay_factor = decay_factor
        self.learning_rate = learning_rate
        self.q_table = np.zeros((21,10,2))
        
    def get_next_action(self, state):
        if random.random() < self.exploration_rate: # Explore (gamble) or exploit (greedy)
            return self.random_action()
        else:
            return self.greedy_action(state)

    def greedy_action(self, state):
        return np.argmax(self.q_table[state[0]][state[1]])
    def random_action(self):
        return random.random() > 0.5


    def train(self, old_state, new_state, action, reward):
        
        old_state_prediction = self.q_table[old_state[0]][old_state[1]][action]
        new_state_prediction = self.q_table[new_state[0]][new_state[1]]

        old_state_prediction = ((1-self.learning_rate) * old_state_prediction) + (self.learning_rate * (reward + self.discount * np.amax(new_state_prediction)))

        self.q_table[old_state[0]][old_state[1]][action] = old_state_prediction

    def update(self, old_state, new_state, action, reward):        
        self.train(old_state, new_state, action, reward)
        self.exploration_rate *= self.decay_factor



In [ ]:
import random

class environment:

    def __init__(self, config=None):

        self.config = config
        self.agent = agent(
            discount=self.config['discount'],
            exploration_rate=self.config['exploration_rate'],
            decay_factor=self.config['decay_factor'],
            learning_rate=self.config['learning_rate']
            )
        self.initGame()

    def initGame(self):
        self.state = (0,0) #a tuble first is players total, second is dealers open card.
        self.state[0] += self.hit()
        self.state[0] += self.hit()
        self.state[1] += self.hit()

    def start(self):
        for episode in range(1,self.config['episode']+1):
            self.initGame()
            while True:
                action_to_play = self.agent.get_next_action(self.state)
                new_state, reward = self.play(action_to_play)
                self.agent.update(self.state,new_state,action_to_play,reward)

    def hit(self):
        return random.randint(1,10)

    def play(self,action):
        new_state = (self.state[0],self.state[1])
        reward = 0
        if action:
            new_state[0] += self.hit()
        if new_state[0] > 21:
            reward = -10
        


